# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# Install and import libraries and dataset

In [ ]:
!pip install -U scikit-learn==1.1.0
!pip install Sastrawi
!pip install nlp-id

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for nlp-id: filename=nlp_id-0.1.13.0-py3-none-any.whl size=8074941 sh

In [ ]:
import pandas as pd
import nltk
import re
import string 
import os
import numpy as np

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from gensim.models import FastText

In [ ]:
# df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/ruhutsitompul/tweets_ruhutsitompu\.csv', delimiter = ';')
df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/03__nakula/tweets_03__nakula.csv', delimiter = ';')
# df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/BengkeltanahONE/tweets_BengkeltanahONE.csv', delimiter = ';')

# Data exploration

In [ ]:
# Check data shape
df_tweets.shape

(73246, 16)

In [ ]:
# Data overview
df_tweets.head()

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count
0,0,1593085082968854529,2022-11-17 03:34:16+00:00,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,NaN,NaN,03__nakula,NaN,41979,7033,8134,116299
1,1,1593084250386857985,2022-11-17 03:30:58+00:00,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",NaN,NaN,03__nakula,NaN,41979,7033,8134,116299
2,2,1593081057141673987,2022-11-17 03:18:16+00:00,1593076365091225601,0,0,@anakodok2009 GOBLOG!!,@anakodok2009 GOBLOG!!,1.593079e+18,https://twitter.com/anakodok2009,03__nakula,NaN,41979,7033,8134,116299
3,3,1593077467907923969,2022-11-17 03:04:00+00:00,1593019336309866496,0,0,@mlgstop007 Goblog!!,@mlgstop007 Goblog!!,1.593077e+18,https://twitter.com/mlgstop007,03__nakula,NaN,41979,7033,8134,116299
4,4,1593076365091225601,2022-11-17 02:59:38+00:00,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",NaN,NaN,03__nakula,NaN,41979,7033,8134,116299


In [ ]:
# Check missing values
df_tweets.isnull().sum()

Unnamed: 0                  0
ID                          0
Tweet Time                  0
Convo ID                    0
Likes Count                 0
Retweets Count              0
Tweets                      0
Tweets(rendered)            0
in_reply_to_tweet_id    23627
in_reply_to             23627
User                        0
User Location           73246
Followers Count             0
Followings Count            0
Likes Count.1               0
Tweets Count                0
dtype: int64

In [ ]:
# Drop duplicate tweets
df_tweets.drop_duplicates(subset="Tweets", keep='first', inplace=True)
df_tweets.reset_index()

,index,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count
0,0,0,1593085082968854529,2022-11-17 03:34:16+00:00,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,NaN,NaN,03__nakula,NaN,41979,7033,8134,116299
1,1,1,1593084250386857985,2022-11-17 03:30:58+00:00,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",NaN,NaN,03__nakula,NaN,41979,7033,8134,116299
2,2,2,1593081057141673987,2022-11-17 03:18:16+00:00,1593076365091225601,0,0,@anakodok2009 GOBLOG!!,@anakodok2009 GOBLOG!!,1.593079e+18,https://twitter.com/anakodok2009,03__nakula,NaN,41979,7033,8134,116299
3,3,3,1593077467907923969,2022-11-17 03:04:00+00:00,1593019336309866496,0,0,@mlgstop007 Goblog!!,@mlgstop007 Goblog!!,1.593077e+18,https://twitter.com/mlgstop007,03__nakula,NaN,41979,7033,8134,116299
4,4,4,1593076365091225601,2022-11-17 02:59:38+00:00,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",NaN,NaN,03__nakula,NaN,41979,7033,8134,116299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73241,73241,73241,1118345093654560768,2019-04-17 02:47:02+00:00,1118345093654560768,51,20,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,NaN,NaN,03__nakula,NaN,42003,7035,8119,116216
73242,73242,73242,1118331360588943360,2019-04-17 01:52:28+00:00,1118195655468392448,1,0,@Novalku @_______B1G__k4y @Imnida03 @badonci @...,@Novalku @_______B1G__k4y @Imnida03 @badonci @...,1.118323e+18,https://twitter.com/Novalku,03__nakula,NaN,42003,7035,8119,116216
73243,73243,73243,1118319523113816064,2019-04-17 01:05:26+00:00,1118319523113816064,4,4,Dihimbau untuk pendukung Jokowi jangan ikut se...,Dihimbau untuk pendukung Jokowi jangan ikut se...,NaN,NaN,03__nakula,NaN,42003,7035,8119,116216
73244,73244,73244,1118314844850049024,2019-04-17 00:46:50+00:00,1118308976297254912,0,0,@01_PandixaDz @_______B1G__k4y @BadjaNuswantar...,@01_PandixaDz @_______B1G__k4y @BadjaNuswantar...,1.118314e+18,https://twitter.com/Ana___Dz,03__nakula,NaN,42003,7035,8119,116216


In [ ]:
from nltk import FreqDist

# Function to get top occuring words
def get_freq_word(df_input, column_name):
  text = " ".join(df_input[column_name])
  words = text.split()
  freq_word = FreqDist(words)
  return freq_word

# Function to sort a tuple descendingly
def sort_tuple(tup):
    tup.sort(key = lambda x: x[1], reverse=True)
    return tup

In [ ]:
# Get and sort top occuring words
freq_words_list = list(filter(lambda x: x[1]>=1, get_freq_word(df_tweets, 'Tweets').items()))
sort_tuple(freq_words_list)

[('yang', 17272),
 ('dan', 16539),
 ('di', 14475),
 ('untuk', 6571),
 ('gak', 6036),
 ('dengan', 5397),
 ('nya', 4529),
 ('ada', 4390),
 ('tidak', 3876),
 ('jadi', 3693),
 ('bisa', 3455),
 ('Ganjar', 3331),
 ('@shinichikudo991', 3141),
 ('ke', 3127),
 ('dari', 3102),
 ('ini', 3018),
 ('saya', 3001),
 ('@NinjaCir3ng', 2938),
 ('@P3nj3l4j4h_id', 2933),
 ('akan', 2753),
 ('itu', 2724),
 ('karena', 2661),
 ('juga', 2620),
 ('😂', 2541),
 ('kita', 2429),
 ('😂😂', 2354),
 ('sudah', 2306),
 ('orang', 2283),
 ('@ganjarpranowo', 2267),
 ('Indonesia', 2257),
 ('Erick', 2238),
 ('kamu', 2210),
 ('@P3nj3l4j4h', 2174),
 ('Kalo', 2108),
 ('@Paltiwest', 2069),
 ('mereka', 2050),
 ('ya', 2011),
 ('dalam', 1995),
 ('aja', 1984),
 ('@BadjaNuswantara', 1877),
 ('🤣🤣', 1871),
 ('BUMN', 1857),
 ('🤭', 1844),
 ('harus', 1829),
 ('Thohir', 1817),
 ('agar', 1677),
 ('@_______B1G__k4y', 1673),
 ('kalo', 1653),
 ('@Aryprasetyo85', 1639),
 ('Jawa', 1601),
 ('tapi', 1597),
 ('aku', 1591),
 ('kan', 1577),
 ('#SahabatG

In [ ]:
# Get and sort top hashtags
hashtags_list = list(filter(lambda x: x[0][0][0]=='#', freq_words_list))
sort_tuple(hashtags_list)

[('#SahabatGanjar', 1553),
 ('#Ganjar', 1056),
 ('#GanjarPranowo', 930),
 ('#BangkitBersamaET', 893),
 ('#KPKPanggilZulPAN', 453),
 ('#TolakAnakISISKembali', 452),
 ('#ganjarpranowo', 446),
 ('#Didubutuhkelon', 233),
 ('#sahabatganjar', 228),
 ('#NegeriIntoleran', 167),
 ('#DenganETMerdekaBerdaulat', 142),
 ('#KasihanJokowi', 141),
 ('#ErickThohirSolusiRakyat', 134),
 ('#SAGA', 104),
 ('#PDIPPatuh', 102),
 ('#PolisikanAndreRosiade', 101),
 ('#UltahBuMega', 99),
 ('#ganjarpilihanrakyat', 93),
 ('#PDIPCintaiBumi', 93),
 ('#BobbyAuliaBenahiMedan', 89),
 ('#PraPeradilanTitan', 81),
 ('#BobbyAuliaKeren', 78),
 ('#TolakPembebasanKoruptor', 78),
 ('#RakyatPercayaJokowi', 76),
 ('#PDIP47Tahun', 76),
 ('#MafiaKesehatan', 75),
 ('#JiwasrayaMeradang', 75),
 ('#MedanYess', 73),
 ('#ProsesAndreRosiade', 71),
 ('#ganjar', 66),
 ('#BedahKasus', 62),
 ('#MobileLegends', 57),
 ('#MedanMantul', 57),
 ('#DandaniMedan', 57),
 ('#Deklarasi', 55),
 ('#75MerdekaPDIP', 54),
 ('#JumatBerkah', 53),
 ('#BantengI

In [ ]:
# List of top hashtags (Count >= 2)
hashtags = []

for i, _ in enumerate(hashtags_list):
  if(hashtags_list[i][1]>=2):
    hashtags.append(hashtags_list[i][0])

hashtags_lower = []
for word in hashtags:
  hashtags_lower.append(word.lower())

hashtags_lower

['#sahabatganjar',
 '#ganjar',
 '#ganjarpranowo',
 '#bangkitbersamaet',
 '#kpkpanggilzulpan',
 '#tolakanakisiskembali',
 '#ganjarpranowo',
 '#didubutuhkelon',
 '#sahabatganjar',
 '#negeriintoleran',
 '#denganetmerdekaberdaulat',
 '#kasihanjokowi',
 '#erickthohirsolusirakyat',
 '#saga',
 '#pdippatuh',
 '#polisikanandrerosiade',
 '#ultahbumega',
 '#ganjarpilihanrakyat',
 '#pdipcintaibumi',
 '#bobbyauliabenahimedan',
 '#praperadilantitan',
 '#bobbyauliakeren',
 '#tolakpembebasankoruptor',
 '#rakyatpercayajokowi',
 '#pdip47tahun',
 '#mafiakesehatan',
 '#jiwasrayameradang',
 '#medanyess',
 '#prosesandrerosiade',
 '#ganjar',
 '#bedahkasus',
 '#mobilelegends',
 '#medanmantul',
 '#dandanimedan',
 '#deklarasi',
 '#75merdekapdip',
 '#jumatberkah',
 '#bantenginovasipdip',
 '#umkmcashback',
 '#eh',
 '#isisgotohell',
 '#solusimedan',
 '#pancasilasanubarikita',
 '#cakrawatibangsa',
 '#tempomediaasu',
 '#viruscinta',
 '#jemputsaiddidu',
 '#tetapmega',
 '#pembagianpaketsembako',
 '#jatenggayeng',
 '#k

In [ ]:
# Get and sort top usernames
usernames_list = list(filter(lambda x: x[0][0][0]=='@', freq_words_list))
sort_tuple(usernames_list)

[('@shinichikudo991', 3141),
 ('@NinjaCir3ng', 2938),
 ('@P3nj3l4j4h_id', 2933),
 ('@ganjarpranowo', 2267),
 ('@P3nj3l4j4h', 2174),
 ('@Paltiwest', 2069),
 ('@BadjaNuswantara', 1877),
 ('@_______B1G__k4y', 1673),
 ('@Aryprasetyo85', 1639),
 ('@jokowi', 1194),
 ('@YRadianto', 1155),
 ('@Pertapatua_', 1119),
 ('@arjuno_ireng01', 999),
 ('@AryPrasetyo_85', 987),
 ('@Vivittts', 933),
 ('@PartaiSocmed', 926),
 ('@shda_agatha', 909),
 ('@TheArieAir', 901),
 ('@putri_lior4', 891),
 ('@mochamadarip', 885),
 ('@kangmirdja', 845),
 ('@Rahman_Nashir', 825),
 ('@Hanifah933', 805),
 ('@narkosun', 775),
 ('@wisanggeni_084', 755),
 ('@My_LoveNebe', 754),
 ('@Cebongmilitan1', 727),
 ('@Je_Ly', 717),
 ('@FarelmaxL', 706),
 ('@Namaku_Mei', 686),
 ('@NgkongRoses', 681),
 ('@AtikaFaya', 679),
 ('@Cintada16', 670),
 ('@andri_attitude', 670),
 ('@DivHumas_Polri', 669),
 ('@achzam_prabu', 646),
 ('@DjanChoek', 637),
 ('@WahabisLokal', 631),
 ('@amrudinnejad_', 627),
 ('@ChusnulCh__', 613),
 ('@Widyarenee', 5

In [ ]:
# List of top usernames (Count >= 500)
usernames = []

for i, _ in enumerate(usernames_list):
  if(usernames_list[i][1]>=500):
    usernames.append(usernames_list[i][0])

usernames_lower = []
for word in usernames:
  usernames_lower.append(word.lower())

usernames_lower

['@shinichikudo991',
 '@ninjacir3ng',
 '@p3nj3l4j4h_id',
 '@ganjarpranowo',
 '@p3nj3l4j4h',
 '@paltiwest',
 '@badjanuswantara',
 '@_______b1g__k4y',
 '@aryprasetyo85',
 '@jokowi',
 '@yradianto',
 '@pertapatua_',
 '@arjuno_ireng01',
 '@aryprasetyo_85',
 '@vivittts',
 '@partaisocmed',
 '@shda_agatha',
 '@thearieair',
 '@putri_lior4',
 '@mochamadarip',
 '@kangmirdja',
 '@rahman_nashir',
 '@hanifah933',
 '@narkosun',
 '@wisanggeni_084',
 '@my_lovenebe',
 '@cebongmilitan1',
 '@je_ly',
 '@farelmaxl',
 '@namaku_mei',
 '@ngkongroses',
 '@atikafaya',
 '@cintada16',
 '@andri_attitude',
 '@divhumas_polri',
 '@achzam_prabu',
 '@djanchoek',
 '@wahabislokal',
 '@amrudinnejad_',
 '@chusnulch__',
 '@widyarenee',
 '@andy_nomicin',
 '@dwiyana_dkm',
 '@pujangga1123',
 '@rizieqdivist',
 '@ch_chotimah',
 '@heniunique',
 '@ariandiputra30',
 '@v_stone_kardol']

# Data pre-processing

In [ ]:
# Add custom stopwords and dictionary
custom_stopwords = ['rt'
                    , 'httapis', 'https' 'user', 't', 'co', 'ha', 'shalom', 'ye', 'deh', 'cian', 'sich', 'yang', 'shalom', 'dan',
                    'sudah', 'aku', 'dengan', 'pak', 'ini', 'sip', 'ka', 'ini', 'utk', 'eh', 'nih', 'sudah', 'bpk', 'dgn', 'jgn',
                    'nie', 'terusssssss', 'ma', 'u', 'tdk', 'ra', 'le', 'ba', 'biar', 'terussssssss', 'ta', 'm', 'ha', 'deh',
                    'mk', 'kacian', 'yg', 'dan', 'shalom', 'tuhan', 'yesus', 'haleluya', 'puji',
                    'httpurl', 'HTTPURL',
                    '@USER', 'USER', 'user',
                    'tertawa', 'gak', 'wkwkwk', 'sdh'
                    ]

custom_dict = {
               }

In [ ]:
# Add top hashtags and usernames to a dictionary
top_hashtags_usernames = {'#SahabatGanjar': 'sahabat ganjar',
                          '#Ganjar': 'ganjar',
                          '#GanjarPranowo': 'ganjar pranowo',
                          '#Ganjar': 'ganjar',
                          '#BangkitBersamaET': 'bangkit bersama erick thohir',
                          '#KPKPanggilZulPAN': 'kpk panggil zulpan',
                          '#TolakAnakISISKembali': 'tolak aksi isis kembali',
                          '#ganjarpranowo': 'ganjar pranowo',
                          '#Didubutuhkelon': 'didu butuh kelon',
                          '#sahabatganjar': 'sahabat ganjar',
                          '#NegeriIntoleran': 'negeri intoleran',
                          '#DenganETMerdekaBerdaulat': 'dengan erick thohir merdeka berdaulat',
                          '#KasihanJokowi': 'kasihan jokowi',
                          '#ErickThohirSolusiRakyat': 'erick thohir solusi rakyat',
                          '#SAGA': 'saga',
                          '#PDIPPatuh': 'pdip patuh',
                          '#PolisikanAndreRosiade': 'polisikan andre rosiade',
                          '#ErickThohirSolusiRakyat': 'erick thohir solusi rakyat',

                          '@jokowi': 'jokowi'
                          }

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from nlp_id.stopword import StopWord 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Combine stopwords
stopwords_sastrawi = StopWordRemoverFactory()
stopwords_nlpid = StopWord() 
stopwords_nltk = stopwords.words('indonesian')
stopwords_github = list(np.array(pd.read_csv("/content/drive/MyDrive/TA/stopwords.txt", header=None).values).squeeze())
more_stopword = custom_stopwords
data_stopword = stopwords_sastrawi.get_stop_words() + stopwords_nlpid.get_stopword() + stopwords_github + stopwords_nltk + more_stopword 
data_stopword = list(set(data_stopword))

# Only use 'rt' as stopwords
data_stopword = list(set(data_stopword))

# Combine slang dictionary
import json
with open('/content/drive/MyDrive/TA/slang.txt') as f:
    data = f.read()
data_slang = json.loads(data) # reconstructing the data as a dictionary

print(data_slang)
more_dict = custom_dict
update_dict = data_slang.update(more_dict)

{'@': 'di', 'abis': 'habis', 'ad': 'ada', 'adlh': 'adalah', 'afaik': 'as far as i know', 'ahaha': 'haha', 'aj': 'saja', 'ajep-ajep': 'dunia gemerlap', 'ak': 'saya', 'akika': 'aku', 'akkoh': 'aku', 'akuwh': 'aku', 'alay': 'norak', 'alow': 'halo', 'ambilin': 'ambilkan', 'ancur': 'hancur', 'anjrit': 'anjing', 'anter': 'antar', 'ap2': 'apa-apa', 'apasih': 'apa sih', 'apes': 'sial', 'aps': 'apa', 'aq': 'saya', 'aquwh': 'aku', 'asbun': 'asal bunyi', 'aseekk': 'asyik', 'asekk': 'asyik', 'asem': 'asam', 'aspal': 'asli tetapi palsu', 'astul': 'asal tulis', 'ato': 'atau', 'au ah': 'tidak mau tahu', 'awak': 'saya', 'ay': 'sayang', 'ayank': 'sayang', 'b4': 'sebelum', 'bakalan': 'akan', 'bandes': 'bantuan desa', 'bangedh': 'banget', 'banpol': 'bantuan polisi', 'banpur': 'bantuan tempur', 'basbang': 'basi', 'bcanda': 'bercanda', 'bdg': 'bandung', 'begajulan': 'nakal', 'beliin': 'belikan', 'bencong': 'banci', 'bentar': 'sebentar', 'ber3': 'bertiga', 'beresin': 'membereskan', 'bete': 'bosan', 'beud': 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_stopword

['toh',
 'mendatangi',
 'kenapa',
 'tadi-tadi',
 'jumlahnya',
 'lur',
 'jumat',
 'beliau',
 'diberikannya',
 'mengetahui',
 'pantas',
 'cuma',
 'paparnya',
 'begini-gini',
 'sejak',
 'cieleh',
 'tengah',
 'naik',
 'ingin',
 'datang',
 'kelima',
 'segamblang-gamblangnya',
 'tatkala',
 'ungkapnya',
 'ternyata',
 'diibaratkannya',
 'itulah',
 'halah',
 'alhamdulillah',
 'dimintai',
 'dijamu',
 'tuhan',
 'berturut',
 'pekan',
 'serupa',
 'cukup-cukup',
 'bagian',
 'bahasanya',
 'balikan',
 'yesus',
 'gue-gue',
 'bagaimanakah',
 'mengucapkannya',
 'semaraknya',
 'mungkinlah',
 'seumpamanya',
 'sehari-seharinya',
 'sehingga',
 'disebutkan',
 'sedemikian',
 'terhadap',
 'pas',
 'akhiri',
 'terlihat',
 'itu-begitu',
 'sederajatnya',
 'sebagaimananya',
 'tersebut',
 'siapa-siapa',
 'saat',
 'sebagai',
 'tahun',
 'tandasnya',
 'inikah',
 'mampukah',
 'sebaliknya',
 'dipergunakan',
 'the',
 'bawah',
 'dkk',
 'seluruhnya',
 'lebih-lebih',
 'seraya',
 'tutur',
 'santailah',
 'rt',
 'bisa-bisa',
 's

In [ ]:
import spacy

# Adding stopwords
nlp = spacy.blank('id')
for word in data_stopword:
    nlp.vocab[word].is_stop = True

In [ ]:
import gensim
import re
from nlp_id.tokenizer import Tokenizer
tokenizer = Tokenizer()

# Function to preprocess a text
def preprocess(text):
  # Case folding
  text = text.lower()
  
  # Change HTML entities
  text = text.replace('&amp;', 'dan')
  text = text.replace('&gt;', 'lebih dari')
  text = text.replace('&lt;', 'kurang dari')

  # Change top occuring hashtags and mentions
  for word, replacement in top_hashtags_usernames.items():
    text = text.replace(word, replacement)
  
  # Remove url
  text = re.sub(r'http\S+', 'HTTPURL', text)
  
  # Remove HTML tags
  text = re.sub(r'<.*?>', ' ', text)
  
  # Remove hashtags
  text = re.sub(r'#\w+', ' ', text)
  
  # Replace @mentions with '@USER'
  text = re.sub(r'@\w+', '@USER', text)

  # Remove non-letter characters
  text = re.sub('[^a-zA-z]', ' ', text)

  # Remove excess space
  text = re.sub(' +', ' ', text)
  text = text.strip()

  result = []
  word_token = tokenizer.tokenize(text) # Tokenize words
  for word in word_token:
    word = word.strip().lower() # Case Folding to Lower Case
    if word in data_slang:
      word = data_slang[word]
    if word not in data_stopword: # Stopwords removal
      result.append(word)
    else:
      continue

  if(len(result) < 4):
    result_sentence = None  # Drop words with less than 4 characters 
  else:
    result_sentence = " ".join(result).strip() # Concatenate the result of preprocessing

  return result_sentence  

In [ ]:
# Pre-process tweets
df_tweets['clean_tweets'] = [preprocess(text) for text in df_tweets['Tweets']]
df_tweets['clean_tweets']

0        presiden jokowi optimis kereta cepat jakarta b...
1        menteri bumn erick thohir uji coba kereta cepa...
2                                                     None
3                                                     None
4        gubernur jawa ganjar pranowo tinjau lokasi tan...
                               ...                        
73241    bismillah pilih jokowi kepentingan kepuasan pr...
73242                                                 None
73243    dihimbau pendukung jokowi sebar transkrip wa k...
73244                                                 None
73245                                                 None
Name: clean_tweets, Length: 73246, dtype: object

In [ ]:
# Check missing values
df_tweets['clean_tweets'].isnull().sum()

38761

In [ ]:
# Drop missing values
df_tweets.dropna(axis=0, how='any', inplace=True, subset=['clean_tweets'])
df_tweets.shape

(34485, 17)

In [ ]:
df_tweets.reset_index

<bound method DataFrame.reset_index of        Unnamed: 0                   ID                 Tweet Time  \
0               0  1593085082968854529  2022-11-17 03:34:16+00:00   
1               1  1593084250386857985  2022-11-17 03:30:58+00:00   
4               4  1593076365091225601  2022-11-17 02:59:38+00:00   
5               5  1593075471469510657  2022-11-17 02:56:04+00:00   
7               7  1593066718976307200  2022-11-17 02:21:18+00:00   
...           ...                  ...                        ...   
73226       73226  1118431073442992128  2019-04-17 08:28:42+00:00   
73227       73227  1118428015061426176  2019-04-17 08:16:32+00:00   
73236       73236  1118365021564743680  2019-04-17 04:06:14+00:00   
73241       73241  1118345093654560768  2019-04-17 02:47:02+00:00   
73243       73243  1118319523113816064  2019-04-17 01:05:26+00:00   

                  Convo ID  Likes Count  Retweets Count  \
0      1593085082968854529            3               1   
1      1593084

In [ ]:
from nlp_id.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer()

# Lemmatize cleaned tweets
df_tweets['clean_tweets'] = [lemmatizer.lemmatize(text) for text in df_tweets['clean_tweets']]
df_tweets['clean_tweets']

0        presiden jokowi optimis kereta cepat jakarta b...
1        menteri bumn erick thohir uji coba kereta cepa...
4        gubernur jawa ganjar pranowo tinjau lokasi tan...
5                             kocak muji anggap cibir blok
7        bangkrut kepala yang ekonomi indonesia tumbuh ...
                               ...                        
73226    quick count jokowi prabowo exit poll versi bpn...
73227                           hasil jokowi hasil prabowo
73236                   coblos bareng sayang salam periode
73241    bismillah pilih jokowi penting puas pribadi ta...
73243    dihimbau dukung jokowi sebar transkrip wa bena...
Name: clean_tweets, Length: 34485, dtype: object

In [ ]:
# Get tweet length
df_tweets['clean_tweets_len'] = df_tweets['clean_tweets'].apply(lambda n: len(n.split()))
df_tweets[['clean_tweets', 'clean_tweets_len']]

,clean_tweets,clean_tweets_len
0,presiden jokowi optimis kereta cepat jakarta b...,17
1,menteri bumn erick thohir uji coba kereta cepa...,19
4,gubernur jawa ganjar pranowo tinjau lokasi tan...,16
5,kocak muji anggap cibir blok,5
7,bangkrut kepala yang ekonomi indonesia tumbuh ...,10
...,...,...
73226,quick count jokowi prabowo exit poll versi bpn...,20
73227,hasil jokowi hasil prabowo,4
73236,coblos bareng sayang salam periode,5
73241,bismillah pilih jokowi penting puas pribadi ta...,15


In [ ]:
# Drop data if tweet length is lower than 4
df_tweets = df_tweets.drop(df_tweets[df_tweets.clean_tweets_len < 4].index)
df_tweets.reset_index()

,index,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count,clean_tweets,clean_tweets_len
0,0,0,1593085082968854529,2022-11-17 03:34:16+00:00,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,NaN,NaN,03__nakula,NaN,41979,7033,8134,116299,presiden jokowi optimis kereta cepat jakarta b...,17
1,1,1,1593084250386857985,2022-11-17 03:30:58+00:00,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",NaN,NaN,03__nakula,NaN,41979,7033,8134,116299,menteri bumn erick thohir uji coba kereta cepa...,19
2,4,4,1593076365091225601,2022-11-17 02:59:38+00:00,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",NaN,NaN,03__nakula,NaN,41979,7033,8134,116299,gubernur jawa ganjar pranowo tinjau lokasi tan...,16
3,5,5,1593075471469510657,2022-11-17 02:56:04+00:00,1593052140120858624,0,0,"@the_schloss Kocak lo, banyak yang muji gitu k...","@the_schloss Kocak lo, banyak yang muji gitu k...",1.593070e+18,https://twitter.com/the_schloss,03__nakula,NaN,41979,7033,8134,116299,kocak muji anggap cibir blok,5
4,7,7,1593066718976307200,2022-11-17 02:21:18+00:00,1593019336309866496,0,0,"@niscalarasyid1 Bangkrut pala lo peyang, ekono...","@niscalarasyid1 Bangkrut pala lo peyang, ekono...",1.593053e+18,https://twitter.com/niscalarasyid1,03__nakula,NaN,41979,7033,8134,116299,bangkrut kepala yang ekonomi indonesia tumbuh ...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34479,73226,73226,1118431073442992128,2019-04-17 08:28:42+00:00,1118431073442992128,22,10,"Quick count 55,05% Jokowi \nDan prabowo 44,95%...","Quick count 55,05% Jokowi \nDan prabowo 44,95%...",NaN,NaN,03__nakula,NaN,42003,7035,8119,116216,quick count jokowi prabowo exit poll versi bpn...,20
34480,73227,73227,1118428015061426176,2019-04-17 08:16:32+00:00,1118428015061426176,3,3,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,NaN,NaN,03__nakula,NaN,42003,7035,8119,116216,hasil jokowi hasil prabowo,4
34481,73236,73236,1118365021564743680,2019-04-17 04:06:14+00:00,1118365021564743680,1,0,Nyoblos nya bareng kesayangan 💞\n\nSalam ✌ per...,Nyoblos nya bareng kesayangan 💞\n\nSalam ✌ per...,NaN,NaN,03__nakula,NaN,42003,7035,8119,116216,coblos bareng sayang salam periode,5
34482,73241,73241,1118345093654560768,2019-04-17 02:47:02+00:00,1118345093654560768,51,20,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,NaN,NaN,03__nakula,NaN,42003,7035,8119,116216,bismillah pilih jokowi penting puas pribadi ta...,15


In [ ]:
df_tweets = df_tweets.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
df_tweets['Tweet Time'] = df_tweets['Tweet Time'].astype('datetime64')

In [ ]:
df_tweets.sort_values(by='Tweet Time',ascending=False, inplace=True)

In [ ]:
df_tweets.drop(columns=['Unnamed: 0'])

,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count,clean_tweets,clean_tweets_len
0,1593085082968854529,2022-11-17 03:34:16,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,nan,nan,03__nakula,nan,41979,7033,8134,116299,presiden jokowi optimis kereta cepat jakarta b...,17
1,1593084250386857985,2022-11-17 03:30:58,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",nan,nan,03__nakula,nan,41979,7033,8134,116299,menteri bumn erick thohir uji coba kereta cepa...,19
4,1593076365091225601,2022-11-17 02:59:38,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",nan,nan,03__nakula,nan,41979,7033,8134,116299,gubernur jawa ganjar pranowo tinjau lokasi tan...,16
5,1593075471469510657,2022-11-17 02:56:04,1593052140120858624,0,0,"@the_schloss Kocak lo, banyak yang muji gitu k...","@the_schloss Kocak lo, banyak yang muji gitu k...",1.5930697801307505e+18,https://twitter.com/the_schloss,03__nakula,nan,41979,7033,8134,116299,kocak muji anggap cibir blok,5
7,1593066718976307200,2022-11-17 02:21:18,1593019336309866496,0,0,"@niscalarasyid1 Bangkrut pala lo peyang, ekono...","@niscalarasyid1 Bangkrut pala lo peyang, ekono...",1.593052770156032e+18,https://twitter.com/niscalarasyid1,03__nakula,nan,41979,7033,8134,116299,bangkrut kepala yang ekonomi indonesia tumbuh ...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73226,1118431073442992128,2019-04-17 08:28:42,1118431073442992128,22,10,"Quick count 55,05% Jokowi \nDan prabowo 44,95%...","Quick count 55,05% Jokowi \nDan prabowo 44,95%...",nan,nan,03__nakula,nan,42003,7035,8119,116216,quick count jokowi prabowo exit poll versi bpn...,20
73227,1118428015061426176,2019-04-17 08:16:32,1118428015061426176,3,3,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,nan,nan,03__nakula,nan,42003,7035,8119,116216,hasil jokowi hasil prabowo,4
73236,1118365021564743680,2019-04-17 04:06:14,1118365021564743680,1,0,Nyoblos nya bareng kesayangan \n\nSalam perio...,Nyoblos nya bareng kesayangan \n\nSalam perio...,nan,nan,03__nakula,nan,42003,7035,8119,116216,coblos bareng sayang salam periode,5
73241,1118345093654560768,2019-04-17 02:47:02,1118345093654560768,51,20,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,nan,nan,03__nakula,nan,42003,7035,8119,116216,bismillah pilih jokowi penting puas pribadi ta...,15


In [ ]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34484 entries, 0 to 73243
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            34484 non-null  object        
 1   ID                    34484 non-null  object        
 2   Tweet Time            34484 non-null  datetime64[ns]
 3   Convo ID              34484 non-null  object        
 4   Likes Count           34484 non-null  object        
 5   Retweets Count        34484 non-null  object        
 6   Tweets                34484 non-null  object        
 7   Tweets(rendered)      34484 non-null  object        
 8   in_reply_to_tweet_id  34484 non-null  object        
 9   in_reply_to           34484 non-null  object        
 10  User                  34484 non-null  object        
 11  User Location         34484 non-null  object        
 12  Followers Count       34484 non-null  object        
 13  Followings Count

In [ ]:
# Replace empty tweet with NaN
df_tweets['clean_tweets'] = df_tweets['clean_tweets'].replace(r'^\s*$', np.NaN, regex=True)

In [ ]:
df_tweets = df_tweets[df_tweets['clean_tweets'].notna()]

In [ ]:
df_tweets.sort_values(by=['Tweet Time'], ascending=False, inplace=True)
df_tweets.reset_index(drop=True)

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count,clean_tweets,clean_tweets_len
0,0,1593085082968854529,2022-11-17 03:34:16,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,nan,nan,03__nakula,nan,41979,7033,8134,116299,presiden jokowi optimis kereta cepat jakarta b...,17
1,1,1593084250386857985,2022-11-17 03:30:58,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",nan,nan,03__nakula,nan,41979,7033,8134,116299,menteri bumn erick thohir uji coba kereta cepa...,19
2,4,1593076365091225601,2022-11-17 02:59:38,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",nan,nan,03__nakula,nan,41979,7033,8134,116299,gubernur jawa ganjar pranowo tinjau lokasi tan...,16
3,5,1593075471469510657,2022-11-17 02:56:04,1593052140120858624,0,0,"@the_schloss Kocak lo, banyak yang muji gitu k...","@the_schloss Kocak lo, banyak yang muji gitu k...",1.5930697801307505e+18,https://twitter.com/the_schloss,03__nakula,nan,41979,7033,8134,116299,kocak muji anggap cibir blok,5
4,7,1593066718976307200,2022-11-17 02:21:18,1593019336309866496,0,0,"@niscalarasyid1 Bangkrut pala lo peyang, ekono...","@niscalarasyid1 Bangkrut pala lo peyang, ekono...",1.593052770156032e+18,https://twitter.com/niscalarasyid1,03__nakula,nan,41979,7033,8134,116299,bangkrut kepala yang ekonomi indonesia tumbuh ...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34479,73226,1118431073442992128,2019-04-17 08:28:42,1118431073442992128,22,10,"Quick count 55,05% Jokowi \nDan prabowo 44,95%...","Quick count 55,05% Jokowi \nDan prabowo 44,95%...",nan,nan,03__nakula,nan,42003,7035,8119,116216,quick count jokowi prabowo exit poll versi bpn...,20
34480,73227,1118428015061426176,2019-04-17 08:16:32,1118428015061426176,3,3,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,nan,nan,03__nakula,nan,42003,7035,8119,116216,hasil jokowi hasil prabowo,4
34481,73236,1118365021564743680,2019-04-17 04:06:14,1118365021564743680,1,0,Nyoblos nya bareng kesayangan \n\nSalam perio...,Nyoblos nya bareng kesayangan \n\nSalam perio...,nan,nan,03__nakula,nan,42003,7035,8119,116216,coblos bareng sayang salam periode,5
34482,73241,1118345093654560768,2019-04-17 02:47:02,1118345093654560768,51,20,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,nan,nan,03__nakula,nan,42003,7035,8119,116216,bismillah pilih jokowi penting puas pribadi ta...,15


In [ ]:
df_tweets['Month'] = df_tweets['Tweet Time'].dt.month_name()
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count,clean_tweets,clean_tweets_len,Month
0,0,1593085082968854529,2022-11-17 03:34:16,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,nan,nan,03__nakula,nan,41979,7033,8134,116299,presiden jokowi optimis kereta cepat jakarta b...,17,November
1,1,1593084250386857985,2022-11-17 03:30:58,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",nan,nan,03__nakula,nan,41979,7033,8134,116299,menteri bumn erick thohir uji coba kereta cepa...,19,November
4,4,1593076365091225601,2022-11-17 02:59:38,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",nan,nan,03__nakula,nan,41979,7033,8134,116299,gubernur jawa ganjar pranowo tinjau lokasi tan...,16,November
5,5,1593075471469510657,2022-11-17 02:56:04,1593052140120858624,0,0,"@the_schloss Kocak lo, banyak yang muji gitu k...","@the_schloss Kocak lo, banyak yang muji gitu k...",1.5930697801307505e+18,https://twitter.com/the_schloss,03__nakula,nan,41979,7033,8134,116299,kocak muji anggap cibir blok,5,November
7,7,1593066718976307200,2022-11-17 02:21:18,1593019336309866496,0,0,"@niscalarasyid1 Bangkrut pala lo peyang, ekono...","@niscalarasyid1 Bangkrut pala lo peyang, ekono...",1.593052770156032e+18,https://twitter.com/niscalarasyid1,03__nakula,nan,41979,7033,8134,116299,bangkrut kepala yang ekonomi indonesia tumbuh ...,10,November
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73226,73226,1118431073442992128,2019-04-17 08:28:42,1118431073442992128,22,10,"Quick count 55,05% Jokowi \nDan prabowo 44,95%...","Quick count 55,05% Jokowi \nDan prabowo 44,95%...",nan,nan,03__nakula,nan,42003,7035,8119,116216,quick count jokowi prabowo exit poll versi bpn...,20,April
73227,73227,1118428015061426176,2019-04-17 08:16:32,1118428015061426176,3,3,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,nan,nan,03__nakula,nan,42003,7035,8119,116216,hasil jokowi hasil prabowo,4,April
73236,73236,1118365021564743680,2019-04-17 04:06:14,1118365021564743680,1,0,Nyoblos nya bareng kesayangan \n\nSalam perio...,Nyoblos nya bareng kesayangan \n\nSalam perio...,nan,nan,03__nakula,nan,42003,7035,8119,116216,coblos bareng sayang salam periode,5,April
73241,73241,1118345093654560768,2019-04-17 02:47:02,1118345093654560768,51,20,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,nan,nan,03__nakula,nan,42003,7035,8119,116216,bismillah pilih jokowi penting puas pribadi ta...,15,April


In [ ]:
tweets_by_month = {}
for month in df_tweets['Month'].unique():
    tweets_by_month[month] = df_tweets[(df_tweets.Month == month)]

In [ ]:
tweets_by_month['June']

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count,clean_tweets,clean_tweets_len,Month
5743,5743,1542654870901075968,2022-06-30 23:42:37,1542654870901075968,29,7,"Ibu adalah malaikat buat anaknya, ibu selalu m...","Ibu adalah malaikat buat anaknya, ibu selalu m...",nan,nan,03__nakula,nan,41984,7032,8136,116303,malaikat anak cinta tulus anak apa rasa menter...,20,June
5745,5745,1542647697009545217,2022-06-30 23:14:06,1542647697009545217,426,56,Usai pertemuan presiden Jokowi dengan presiden...,Usai pertemuan presiden Jokowi dengan presiden...,nan,nan,03__nakula,nan,41984,7032,8136,116303,temu presiden jokowi presiden ukraina rusia ta...,20,June
5746,5746,1542641171838578690,2022-06-30 22:48:10,1542641171838578690,274,46,Presiden Jokowi dan Presiden Vladimir Putin me...,Presiden Jokowi dan Presiden Vladimir Putin me...,nan,nan,03__nakula,nan,41984,7032,8136,116303,presiden jokowi presiden vladimir putin baca p...,22,June
5747,5747,1542635036045783041,2022-06-30 22:23:48,1542635036045783041,554,74,Presiden Jokowi beserta rombongan tiba di Krem...,Presiden Jokowi beserta rombongan tiba di Krem...,nan,nan,03__nakula,nan,41984,7032,8136,116303,presiden jokowi rombong kremlin adakan temu pr...,19,June
5748,5748,1542506291305713666,2022-06-30 13:52:12,1542506291305713666,275,70,Tolong dicatat siapa aja yang menolak kebijaka...,Tolong dicatat siapa aja yang menolak kebijaka...,nan,nan,03__nakula,nan,41984,7032,8136,116303,catat tolak bijak anies baswedan masuk nama da...,18,June
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71307,71307,1134674573016719360,2019-06-01 04:14:34,1134674573016719360,148,37,Innalilahi wainnailaihi roji'un.\nSemoga ibu A...,Innalilahi wainnailaihi roji'un.\nSemoga ibu A...,nan,nan,03__nakula,nan,42003,7035,8119,116215,innalilahi wainnailaihi roji un moga ani yudho...,22,June
71308,71308,1134673261902458880,2019-06-01 04:09:21,1134603757960830976,0,0,@Vivittts @shda_agatha Wkwkkwkwkw \nUntung keb...,@Vivittts @shda_agatha Wkwkkwkwkw \nUntung keb...,1.1346728669416858e+18,https://twitter.com/Vivittts,03__nakula,nan,42003,7035,8119,116215,wkwkkwkwkw untung lanjur kabur lempar bom molo...,9,June
71310,71310,1134672433007325185,2019-06-01 04:06:03,1134603757960830976,0,0,@Vivittts Wkwkwkw \nDasar cebong garis keras,@Vivittts Wkwkwkw \nDasar cebong garis keras,1.1346677918505984e+18,https://twitter.com/Vivittts,03__nakula,nan,42003,7035,8119,116215,wkwkwkw dasar cebong garis keras,5,June
71314,71314,1134631459417780224,2019-06-01 01:23:15,1134622974085017600,1,0,@Budiawan1985 @SBYudhoyono @AgusYudhoyono @Bad...,@Budiawan1985 @SBYudhoyono @AgusYudhoyono @Bad...,1.134625321569796e+18,https://twitter.com/Budiawan1985,03__nakula,nan,42003,7035,8119,116215,amin rabbal alamin terima kasih doa,6,June


In [ ]:
count = df_tweets['clean_tweets'].str.split().str.len()

In [ ]:
df_tweets = df_tweets[~(count==1)]

In [ ]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34484 entries, 0 to 73243
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Unnamed: 0            34484 non-null  object        
 1   ID                    34484 non-null  object        
 2   Tweet Time            34484 non-null  datetime64[ns]
 3   Convo ID              34484 non-null  object        
 4   Likes Count           34484 non-null  object        
 5   Retweets Count        34484 non-null  object        
 6   Tweets                34484 non-null  object        
 7   Tweets(rendered)      34484 non-null  object        
 8   in_reply_to_tweet_id  34484 non-null  object        
 9   in_reply_to           34484 non-null  object        
 10  User                  34484 non-null  object        
 11  User Location         34484 non-null  object        
 12  Followers Count       34484 non-null  object        
 13  Followings Count

In [ ]:
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count,clean_tweets,clean_tweets_len,Month
0,0,1593085082968854529,2022-11-17 03:34:16,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,nan,nan,03__nakula,nan,41979,7033,8134,116299,presiden jokowi optimis kereta cepat jakarta b...,17,November
1,1,1593084250386857985,2022-11-17 03:30:58,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",nan,nan,03__nakula,nan,41979,7033,8134,116299,menteri bumn erick thohir uji coba kereta cepa...,19,November
4,4,1593076365091225601,2022-11-17 02:59:38,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",nan,nan,03__nakula,nan,41979,7033,8134,116299,gubernur jawa ganjar pranowo tinjau lokasi tan...,16,November
5,5,1593075471469510657,2022-11-17 02:56:04,1593052140120858624,0,0,"@the_schloss Kocak lo, banyak yang muji gitu k...","@the_schloss Kocak lo, banyak yang muji gitu k...",1.5930697801307505e+18,https://twitter.com/the_schloss,03__nakula,nan,41979,7033,8134,116299,kocak muji anggap cibir blok,5,November
7,7,1593066718976307200,2022-11-17 02:21:18,1593019336309866496,0,0,"@niscalarasyid1 Bangkrut pala lo peyang, ekono...","@niscalarasyid1 Bangkrut pala lo peyang, ekono...",1.593052770156032e+18,https://twitter.com/niscalarasyid1,03__nakula,nan,41979,7033,8134,116299,bangkrut kepala yang ekonomi indonesia tumbuh ...,10,November
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73226,73226,1118431073442992128,2019-04-17 08:28:42,1118431073442992128,22,10,"Quick count 55,05% Jokowi \nDan prabowo 44,95%...","Quick count 55,05% Jokowi \nDan prabowo 44,95%...",nan,nan,03__nakula,nan,42003,7035,8119,116216,quick count jokowi prabowo exit poll versi bpn...,20,April
73227,73227,1118428015061426176,2019-04-17 08:16:32,1118428015061426176,3,3,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,nan,nan,03__nakula,nan,42003,7035,8119,116216,hasil jokowi hasil prabowo,4,April
73236,73236,1118365021564743680,2019-04-17 04:06:14,1118365021564743680,1,0,Nyoblos nya bareng kesayangan \n\nSalam perio...,Nyoblos nya bareng kesayangan \n\nSalam perio...,nan,nan,03__nakula,nan,42003,7035,8119,116216,coblos bareng sayang salam periode,5,April
73241,73241,1118345093654560768,2019-04-17 02:47:02,1118345093654560768,51,20,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,nan,nan,03__nakula,nan,42003,7035,8119,116216,bismillah pilih jokowi penting puas pribadi ta...,15,April


In [ ]:
df_tweets.drop_duplicates(subset=['ID'], inplace=True)
df_tweets.drop(columns=['Unnamed: 0'])

,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,User,User Location,Followers Count,Followings Count,Likes Count.1,Tweets Count,clean_tweets,clean_tweets_len,Month
0,1593085082968854529,2022-11-17 03:34:16,1593085082968854529,3,1,Presiden Jokowi optimis kereta cepat jakarta-b...,Presiden Jokowi optimis kereta cepat jakarta-b...,nan,nan,03__nakula,nan,41979,7033,8134,116299,presiden jokowi optimis kereta cepat jakarta b...,17,November
1,1593084250386857985,2022-11-17 03:30:58,1593084250386857985,3,2,"Menteri BUMN Erick Thohir mengatakan, uji coba...","Menteri BUMN Erick Thohir mengatakan, uji coba...",nan,nan,03__nakula,nan,41979,7033,8134,116299,menteri bumn erick thohir uji coba kereta cepa...,19,November
4,1593076365091225601,2022-11-17 02:59:38,1593076365091225601,27,6,"Gubernur Jawa tengah, bapak Ganjar pranowo tin...","Gubernur Jawa tengah, bapak Ganjar pranowo tin...",nan,nan,03__nakula,nan,41979,7033,8134,116299,gubernur jawa ganjar pranowo tinjau lokasi tan...,16,November
5,1593075471469510657,2022-11-17 02:56:04,1593052140120858624,0,0,"@the_schloss Kocak lo, banyak yang muji gitu k...","@the_schloss Kocak lo, banyak yang muji gitu k...",1.5930697801307505e+18,https://twitter.com/the_schloss,03__nakula,nan,41979,7033,8134,116299,kocak muji anggap cibir blok,5,November
7,1593066718976307200,2022-11-17 02:21:18,1593019336309866496,0,0,"@niscalarasyid1 Bangkrut pala lo peyang, ekono...","@niscalarasyid1 Bangkrut pala lo peyang, ekono...",1.593052770156032e+18,https://twitter.com/niscalarasyid1,03__nakula,nan,41979,7033,8134,116299,bangkrut kepala yang ekonomi indonesia tumbuh ...,10,November
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73226,1118431073442992128,2019-04-17 08:28:42,1118431073442992128,22,10,"Quick count 55,05% Jokowi \nDan prabowo 44,95%...","Quick count 55,05% Jokowi \nDan prabowo 44,95%...",nan,nan,03__nakula,nan,42003,7035,8119,116216,quick count jokowi prabowo exit poll versi bpn...,20,April
73227,1118428015061426176,2019-04-17 08:16:32,1118428015061426176,3,3,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,Hasil akhir 62% untuk Jokowi\nHasil akhir 38% ...,nan,nan,03__nakula,nan,42003,7035,8119,116216,hasil jokowi hasil prabowo,4,April
73236,1118365021564743680,2019-04-17 04:06:14,1118365021564743680,1,0,Nyoblos nya bareng kesayangan \n\nSalam perio...,Nyoblos nya bareng kesayangan \n\nSalam perio...,nan,nan,03__nakula,nan,42003,7035,8119,116216,coblos bareng sayang salam periode,5,April
73241,1118345093654560768,2019-04-17 02:47:02,1118345093654560768,51,20,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,Bismillah...\nSaya pilih Jokowi #OrangBaik \nB...,nan,nan,03__nakula,nan,42003,7035,8119,116216,bismillah pilih jokowi penting puas pribadi ta...,15,April


In [ ]:
df_tweets.to_csv('03__nakula_clean.csv')